# **Building Dynamic Models Using the Subclassing API**

Both the Sequential API and the Functional API are declarative: you start by declaring which layers you want to use and how they should be connected, and only then can you start feeding the model some data for training or inference.This has many advantages: the model can easily be saved, cloned, shared, its structure can be displayed and analyzed, the framework can infer shapes and check types, so errors can be caught early.

---

But the flip side is just
that: it’s static. Some models involve loops, varying shapes, conditional branching, and other dynamic behaviors. For such cases, or simply if you prefer a more imperative programming style, the Subclassing API is for you.

---

In [28]:
import tensorflow as tf
from tensorflow import keras

In [29]:
class WideAndDeepModel(keras.models.Model):
  def __init__(self, units=30, activation="relu", **kwargs):
    super().__init__(**kwargs) # handles standard args (e.g., name)
    self.hidden1 = keras.layers.Dense(units, activation=activation)
    self.hidden2 = keras.layers.Dense(units, activation=activation)
    self.main_output = keras.layers.Dense(1)
    self.aux_output = keras.layers.Dense(1)
  def call(self, inputs):
    input_A, input_B = inputs
    hidden1 = self.hidden1(input_B)
    hidden2 = self.hidden2(hidden1)
    concat = keras.layers.concatenate([input_A, hidden2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return main_output, aux_output

model = WideAndDeepModel()


In [30]:
class WideAndDeepModel(keras.models.Model):
  def __init__(self, units=30, activation="relu", **kwargs):
    super().__init__(**kwargs) # handles standard args (e.g., name)
    self.hidden1 = keras.layers.Dense(units, activation=activation)
    self.hidden2 = keras.layers.Dense(units, activation=activation)
    self.main_output = keras.layers.Dense(1)
    self.aux_output = keras.layers.Dense(1)
  def call(self, inputs):
    input_A, input_B = inputs # Corrected: 'input' to 'inputs'
    hidden1 = self.hidden1(input_B)
    hidden2 = self.hidden2(hidden1)
    concat = keras.layers.concatenate([input_A, hidden2])
    main_output = self.main_output(concat)
    aux_output = self.aux_output(hidden2)
    return main_output, aux_output

For example, creating an instance of the following WideAndDeepModel class gives us an equivalent model to the one we just built with the Functional API. You can then compile it, evaluate it and use it to make predictions, exactly like we just did.

---

Simply subclass the Model class, create the layers you need in the constructor, and use them to perform the computations you want in the call( ) method.

---

Saving and Restoring Model

In [31]:
model.save("my_keras_model.h5")

In [32]:
with keras.utils.custom_object_scope({'WideAndDeepModel': WideAndDeepModel}):
  model = keras.models.load_model("/content/my_keras_model.h5")
print("Model loaded successfully!")

Model loaded successfully!


Build and Compile Model

In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load from Excel (offline)
housing = pd.read_excel("/content/fetch_california_housing.xlsx")

housing

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41,6.984127,1.023810,322,2.555556,37.88,-122.23,4.526
1,8.3014,21,6.238137,0.971880,2401,2.109842,37.86,-122.22,3.585
2,7.2574,52,8.288136,1.073446,496,2.802260,37.85,-122.24,3.521
3,5.6431,52,5.817352,1.073059,558,2.547945,37.85,-122.25,3.413
4,3.8462,52,6.281853,1.081081,565,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25,5.045455,1.133333,845,2.560606,39.48,-121.09,0.781
20636,2.5568,18,6.114035,1.315789,356,3.122807,39.49,-121.21,0.771
20637,1.7000,17,5.205543,1.120092,1007,2.325635,39.43,-121.22,0.923
20638,1.8672,18,5.329513,1.171920,741,2.123209,39.43,-121.32,0.847


In [34]:
y = housing['target']
X = housing.drop(columns='target', axis=1)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y)

In [35]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [36]:
y_train

,target
13928,0.48600
13747,1.62000
17999,2.77300
6538,1.37500
4740,5.00001
...,...
20325,5.00001
19916,0.92600
5839,3.41200
8504,2.53600


In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

y_train_scaled = scaler.fit_transform(
    y_train.to_numpy().reshape(-1, 1)
)


In [38]:
y_valid_scaled = scaler.fit_transform(
    y_valid.to_numpy().reshape(-1, 1)
)

In [39]:
import numpy as np

np.isnan(X_train_scaled).any()
np.isnan(y_train).any()
np.isinf(X_train_scaled).any()


np.False_

Build and Compile Model

In [40]:
model = keras.models.Sequential([
    keras.layers.Input(shape=X_train_scaled.shape[1:]),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(30, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer="sgd"
)

In [41]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")
history = model.fit(X_train_scaled, y_train_scaled, epochs=10, callbacks=[checkpoint_cb])

Epoch 1/10
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.8022

363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.8025
Epoch 2/10
350/363 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4701

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.4694
Epoch 3/10
349/363 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3871

363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3867
Epoch 4/10
352/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3641

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3636
Epoch 5/10
361/363 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3577

...

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3189
Epoch 7/10
354/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3017

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3023
Epoch 8/10
344/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3223

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3220
Epoch 9/10
362/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3046

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3045
Epoch 10/10
350/363 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.3038

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3034


Moreover, if you use a validation set during training, you can set
save_best_only=True when creating the ModelCheckpoint. In this case, it will only save your model when its performance on the validation set is the best so far.

---

This way, you do not need to worry about training for too long and overfitting the training set: simply restore the last model saved after training, and this will be the best model on the validation set. This is a simple way to implement early stopping.

---

Saved the best model.

In [42]:
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)
# history = model.fit(X_train_scaled, y_train_scaled, epochs=10, validation_data=(X_valid_scaled, y_valid_scaled), callbacks=[checkpoint_cb])


# model = keras.models.load_model("my_keras_model.h5") # rollback to best model

---

Another way to implement early stopping is to simply use the EarlyStopping call‐
back. It will interrupt training when it measures no progress on the validation set for a number of epochs (defined by the patience argument), and it will optionally roll back to the best model

---

# **Early Stopping**

In [43]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "my_keras_model.keras",
    save_best_only=True
)

early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10,
    restore_best_weights=True
)


In [45]:
history = model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=20,
    validation_data=(X_valid_scaled, y_valid_scaled),
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2055 - val_loss: 0.2220
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2018 - val_loss: 0.2222
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2102 - val_loss: 0.2185
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1987 - val_loss: 0.2266
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2042 - val_loss: 0.2172
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.2106 - val_loss: 0.2269
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2012 - val_loss: 0.2154
Epoch 8/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2138 - val_loss: 0.2138
Epoch 9/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2027 - val_loss: 0.2175
Epoch 10/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2035 - val_loss: 0.2253
Epoch 11/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2093 - val_loss: 0.2127
Epoch 12/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step